In [ ]:
from google.colab import drive

# Mount your Google Drive to access files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from tensorflow.keras.applications import DenseNet201, MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import LossScaleOptimizer
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input as mobilenet_preprocess_input
from sklearn.metrics import accuracy_score

def load_images_and_labels_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(1 if "melanoma" in filename else 0)
    return np.array(images), np.array(labels)

# Load and preprocess images and labels
train_folder = '/content/drive/MyDrive/DL/sc/train'
test_folder = '/content/drive/MyDrive/DL/sc/test'

X_train, y_train = load_images_and_labels_from_folder(os.path.join(train_folder, 'benign'))
X_melanoma, y_melanoma = load_images_and_labels_from_folder(os.path.join(train_folder, 'melanoma'))

# Combine benign and malignant data
X = np.concatenate([X_train, X_melanoma])
y = np.concatenate([np.zeros(len(y_train)), np.ones(len(y_melanoma))])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Augmentation for the training data
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Preprocess images for DenseNet201
X_train_preprocessed_densenet = densenet_preprocess_input(X_train.astype('float32'))
X_val_preprocessed_densenet = densenet_preprocess_input(X_val.astype('float32'))
X_test_preprocessed_densenet = densenet_preprocess_input(X_test.astype('float32'))

# Preprocess images for Mobilenet
X_train_preprocessed_mobilenet = mobilenet_preprocess_input(X_train.astype('float32'))
X_val_preprocessed_mobilenet = mobilenet_preprocess_input(X_val.astype('float32'))
X_test_preprocessed_mobilenet = mobilenet_preprocess_input(X_test.astype('float32'))

In [ ]:

# Load pre-trained DenseNet201 model without the top (fully connected) layer
base_model_densenet = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model for feature extraction
for layer in base_model_densenet.layers:
    layer.trainable = False

# Add your own top layers for classification on top of DenseNet201
x_densenet = layers.GlobalAveragePooling2D()(base_model_densenet.output)
x_densenet = layers.Dense(512, activation='relu')(x_densenet)
x_densenet = layers.Dropout(0.5)(x_densenet)
predictions_densenet = layers.Dense(1, activation='sigmoid')(x_densenet)

# Create the model
model_densenet = Model(inputs=base_model_densenet.input, outputs=predictions_densenet)

# Compile the model
optimizer = Adam(learning_rate=0.0001)  # Adjust the learning rate as needed
optimizer = LossScaleOptimizer(optimizer)
model_densenet.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Use data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train_preprocessed_densenet)

# Train the model with data augmentation
model_densenet.fit(datagen.flow(X_train_preprocessed_densenet, y_train, batch_size=32), epochs=10, validation_data=(X_val_preprocessed_densenet, y_val))



Epoch 1/10
56/56 [==============================] - 718s 13s/step - loss: 0.5631 - accuracy: 0.7065 - val_loss: 0.4367 - val_accuracy: 0.7862
Epoch 2/10
56/56 [==============================] - 663s 12s/step - loss: 0.4484 - accuracy: 0.7941 - val_loss: 0.4016 - val_accuracy: 0.8174
Epoch 3/10
56/56 [==============================] - 758s 14s/step - loss: 0.4008 - accuracy: 0.8231 - val_loss: 0.3781 - val_accuracy: 0.8241
Epoch 4/10
56/56 [==============================] - 652s 12s/step - loss: 0.3860 - accuracy: 0.8315 - val_loss: 0.3645 - val_accuracy: 0.8352
Epoch 5/10
56/56 [==============================] - 728s 13s/step - loss: 0.3707 - accuracy: 0.8404 - val_loss: 0.3596 - val_accuracy: 0.8396
Epoch 6/10
56/56 [==============================] - 707s 13s/step - loss: 0.3671 - accuracy: 0.8304 - val_loss: 0.3601 - val_accuracy: 0.8352
Epoch 7/10
56/56 [==============================] - 659s 12s/step - loss: 0.3552 - accuracy: 0.8471 - val_loss: 0.3520 - val_accuracy: 0.8396
Epoch 

In [ ]:

# Load pre-trained MobileNetV2 model without the top (fully connected) layer
base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model for feature extraction
for layer in base_model_mobilenet.layers:
    layer.trainable = False

# Add your own top layers for classification on top of MobileNetV2
x_mobilenet = layers.GlobalAveragePooling2D()(base_model_mobilenet.output)
x_mobilenet = layers.Dense(512, activation='relu')(x_mobilenet)
x_mobilenet = layers.Dropout(0.5)(x_mobilenet)
predictions_mobilenet = layers.Dense(1, activation='sigmoid')(x_mobilenet)

# Create the model
model_mobilenet = Model(inputs=base_model_mobilenet.input, outputs=predictions_mobilenet)

# Compile the model
model_mobilenet.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_mobilenet.fit(X_train_preprocessed_mobilenet, y_train, epochs=10, batch_size=32, validation_data=(X_val_preprocessed_mobilenet, y_val))

Epoch 1/10
56/56 [==============================] - 97s 2s/step - loss: 0.6303 - accuracy: 0.6769 - val_loss: 0.4830 - val_accuracy: 0.7929
Epoch 2/10
56/56 [==============================] - 86s 2s/step - loss: 0.5015 - accuracy: 0.7645 - val_loss: 0.4626 - val_accuracy: 0.7973
Epoch 3/10
56/56 [==============================] - 87s 2s/step - loss: 0.4635 - accuracy: 0.7773 - val_loss: 0.4273 - val_accuracy: 0.8218
Epoch 4/10
56/56 [==============================] - 90s 2s/step - loss: 0.4105 - accuracy: 0.8153 - val_loss: 0.4197 - val_accuracy: 0.8174
Epoch 5/10
56/56 [==============================] - 89s 2s/step - loss: 0.4044 - accuracy: 0.8225 - val_loss: 0.4204 - val_accuracy: 0.8174
Epoch 6/10
56/56 [==============================] - 87s 2s/step - loss: 0.3883 - accuracy: 0.8259 - val_loss: 0.4013 - val_accuracy: 0.8241
Epoch 7/10
56/56 [==============================] - 98s 2s/step - loss: 0.3643 - accuracy: 0.8421 - val_loss: 0.4109 - val_accuracy: 0.8330
Epoch 8/10
56/56 [==

In [ ]:

# Extract features from DenseNet201
features_densenet_train = model_densenet.predict(X_train_preprocessed_densenet)
features_densenet_val = model_densenet.predict(X_val_preprocessed_densenet)
features_densenet_test = model_densenet.predict(X_test_preprocessed_densenet)

# Extract features from MobileNetV2
features_mobilenet_train = model_mobilenet.predict(X_train_preprocessed_mobilenet)
features_mobilenet_val = model_mobilenet.predict(X_val_preprocessed_mobilenet)
features_mobilenet_test = model_mobilenet.predict(X_test_preprocessed_mobilenet)

# Combine features from both models
X_train_combined = np.concatenate([features_densenet_train, features_mobilenet_train], axis=1)
X_val_combined = np.concatenate([features_densenet_val, features_mobilenet_val], axis=1)
X_test_combined = np.concatenate([features_densenet_test, features_mobilenet_test], axis=1)


13/13 [==============================] - 15s 1s/step


In [ ]:

# Create an SVM model
svm_model = make_pipeline(StandardScaler(), SVC(C=1, kernel='linear', random_state=42))

# Fit the SVM model on the training data
svm_model.fit(X_train_combined, y_train)

# Predict on the validation set
y_val_pred = svm_model.predict(X_val_combined)

# Evaluate accuracy on the validation set
accuracy_val = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy_val}")

# Predict on the test set
y_test_pred = svm_model.predict(X_test_combined)

# Evaluate accuracy on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {accuracy_test}")

Validation Accuracy: 0.8530066815144766
Test Accuracy: 0.8484848484848485
